In [136]:
import numpy as np
import pandas as pd
import math

from sklearn.metrics import ndcg_score, dcg_score

In [172]:
df_articles = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/9f0e8eb4ddd03415fdd4db4a89a2b0d3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/shared_articles.zip')
df_interactions = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/186647c8bd3fdb43b78fbc84ace97aed/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/users_interactions.zip')

In [25]:
df_interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


**Задание 6.1**

Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [26]:
df_articles = df_articles.query('eventType == "CONTENT SHARED"')
df_articles.eventType.count()

3047

In [27]:
# предварительно преобразуем тип данных в ряде столбцов к строкам
df_interactions.personId = df_interactions.personId.astype(str)
df_interactions.contentId = df_interactions.contentId.astype(str)
df_articles.contentId = df_articles.contentId.astype(str)

In [28]:
# Создаём словарь весов различных действий пользователя

event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

**Задание 6.2**

Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [29]:
df_interactions['event_uip'] = df_interactions.eventType.map(event_type)
df_interactions.event_uip.mean().round(2)

1.24

**Задание 6.3**

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [105]:
df_2 = df_interactions.groupby(['personId', 'contentId']).first().reset_index()
(df_2.personId.value_counts() > 4).sum()

1140

**Задание 6.4**

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

In [138]:
df_5 = (df_2.personId.value_counts() > 4).reset_index()
five_or_more = list(df_5[df_5['count'] == True].personId.values)
df_6_4 = df_interactions[df_interactions.personId.isin(five_or_more)]
df_6_4.shape[0]

69868

**Задание 6.5**

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

In [201]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

df_uip = df_6_4.groupby(['personId', 'contentId']).event_uip.sum().apply(smooth_user_preference).reset_index().set_index(['personId', 'contentId'])
df_uip['last_time'] = df_6_4.groupby(['personId', 'contentId']).timestamp.max()

df_uip.reset_index(inplace=True)
df_uip.last_time.mean().round(2)

1470605340.04

**Задание 6.6**

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Сколько объектов попало в обучающую выборку?

In [205]:
df_train = df_uip.query('last_time < 1475519545')
df_test = df_uip.query('last_time >= 1475519545')

df_train.shape[0]

29325

In [194]:
final_df = (
    df_train.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    df_test.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


**Задание 6.7**

Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [220]:
pop_art = df_train.groupby('contentId').event_uip.sum().reset_index().sort_values(by='event_uip', ascending=False).contentId.values
pop_art[0]

'-6783772548752091658'

**Задание 6.8**

Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

In [224]:
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()

In [227]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        pop_art[~np.in1d(pop_art, x)][:top_k]
    )
)

precision('popular').round(3)

0.006